In [1]:
import numpy as np
import pandas as pd
import requests
import datetime
import tqdm
import time
import pickle
from collections import defaultdict
import matplotlib.pyplot as plt

In [50]:
geckterminal_url = "https://api.geckoterminal.com/api/v2"
ton_url = "https://tonapi.io/v2"
stonfi_url = "https://api.ston.fi/v1"
dedust_url = "https://api.dedust.io/v2"
ton_api_key = "AHNEEFDML7QU2FAAAAAF7S4ESSY3QQLM6EDTLGATFSSAG5PQHGQUIFRPZKE7ST6MIVN3B2A"

def get_pickle(path):
    out = None
    with open(path, 'rb') as f:
        out = pickle.load(f)
    return out

## Достаем все пулы на DeDust

### Исследование пулов

In [10]:
url = dedust_url + '/pools'
print(f'get {url}')
pools = requests.get(url)
pools.json()[0]

get https://api.dedust.io/v2/pools


{'address': 'EQDRV4AlGgMx3T1mG3H1tode8_6FXajBMFjd8UHDZ9dqzxbC',
 'lt': '43644368000005',
 'totalSupply': '1000',
 'type': 'volatile',
 'tradeFee': '0.4',
 'assets': [{'type': 'jetton',
   'address': 'EQBiHSLVzoicc-LQPDXJc_rF-WRft0aFihccFduA6fQ1ynYM',
   'metadata': {'name': 'Baron',
    'symbol': 'BARO',
    'image': 'https://assets.dedust.io/images/baro.webp',
    'decimals': 9}},
  {'type': 'jetton',
   'address': 'EQBl3gg6AAdjgjO2ZoNU5Q5EzUIl8XMNZrix8Z5dJmkHUfxI',
   'metadata': {'name': 'Lavandos',
    'symbol': 'LAVE',
    'image': 'https://assets.dedust.io/images/lave.webp',
    'decimals': 9}}],
 'lastPrice': None,
 'reserves': ['825', '122'],
 'stats': {'fees': ['0', '0'], 'volume': ['0', '0']}}

In [11]:
none_supply_pools = 0
none_last_price_pools = 0
types_of_pools = defaultdict(int)
lts = []
supply = []
for pool in pools.json():
    types_of_pools[pool['type']]+=1
    lts.append(int(pool['lt']))
    supply.append(int(pool['totalSupply']))
    if pool['totalSupply'] == '0':
        none_supply_pools += 1
    if pool['lastPrice'] is None:
        none_last_price_pools += 1
print(f'Pools: {len(pools.json())}, with types {list(types_of_pools.items())}')
print(f'Pools with TotalSupply=0: {none_supply_pools}')
print(f'Pools with LastPrice=0: {none_last_price_pools}')

Pools: 19644, with types [('volatile', 19636), ('stable', 8)]
Pools with TotalSupply=0: 1990
Pools with LastPrice=0: 19301


In [12]:
pd.DataFrame(supply).value_counts()

1000                                   14014
0                                       1990
1000000000001000                         124
100000000001000                          103
1000001000                                87
                                       ...  
7423976658023                              1
7304579665423                              1
7281847004526                              1
7233387147520                              1
43990694442740576290346711940594142        1
Length: 2041, dtype: int64

In [13]:
pd.DataFrame(lts).value_counts()

0                 1977
46610847000001      31
46610856000001      19
46610822000001      19
46610833000001      11
                  ... 
45915636000005       1
45915698000005       1
45915822000009       1
45916150000005       1
47008418000011       1
Length: 17364, dtype: int64

In [14]:
for pool in pools.json():
    if pool['type'] == 'stable':
        print(pool['assets'][0]['metadata']['name'], '--',pool['assets'][1]['metadata']['name'] if pool['assets'][1]['metadata'] is not None else pool['assets'][1])

TON Bridge USDT -- TON Bridge USDC
TON Bridge USDT -- Tether USD
Toncoin -- {'type': 'jetton', 'address': 'EQAcHW1VR8AmIlKyC5P9SBVKiVjEwWY1IXpI4w-vXlZKSU7I', 'metadata': None}
Toncoin -- {'type': 'jetton', 'address': 'EQBNo5qAG8I8J6IxGaz15SfQVB-kX98YhKV_mT36Xo5vYxUa', 'metadata': None}
Toncoin -- Ton Whales liquid staking token
Toncoin -- Hipo Staked TON
Toncoin -- Staked TON by bemo
ARBUZ -- ARBUZ


### Достаем все адреса токенов из всех пулов на DeDust

In [18]:
dedust_jettons = []
none_metadata = 0
if pools:
    for i, pool in enumerate(pools.json()):
        if pool['assets'][0]['type'] == 'jetton':
            dedust_jettons.append((pool['assets'][0]['metadata'], pool['assets'][0]['address']))
        if pool['assets'][1]['type'] == 'jetton':
            dedust_jettons.append((pool['assets'][1]['metadata'], pool['assets'][1]['address']))
else:
    print(pools)

In [19]:
dedust_jettons_addr = [j for i,j in dedust_jettons]
dedust_jettons_metadata = [i for i,j in dedust_jettons]
dedust_jettons_addr_dist = set(dedust_jettons_addr)
print("jettons in pools:", len(dedust_jettons))
print("different jettons in pools:", len(set(dedust_jettons_addr)))
print("None metadata jettons in pools:", dedust_jettons_metadata.count(None))

jettons in pools: 21419
different jettons in pools: 18135
None metadata jettons in pools: 18962


In [21]:
# dedust_jettons_to_metadata = {}
dedust_jettons_to_metadata = get_pickle('dicts/dedust_jettons_to_metadata.pkl')

In [22]:
def get_response(jetton):
    response = requests.get(ton_url + f'/jettons/{jetton}')
    if response:
        dedust_jettons_to_metadata[jetton] = response.json()
    else:
        time.sleep(1)
        get_response(jetton)

In [33]:
for jetton in tqdm.tqdm(list(dedust_jettons_addr_dist), total=len(dedust_jettons_addr_dist)):
    if dedust_jettons_to_metadata.get(jetton) is None or dedust_jettons_to_metadata.get(jetton) == {'error': 'rate limit: free tier'}:
        get_response(jetton)

 78%|███████▊  | 14094/18135 [00:01<00:00, 11756.30it/s]


KeyboardInterrupt: 

In [29]:
with open('dicts/dedust_jettons_to_metadata.pkl', 'wb') as f:
    pickle.dump(dedust_jettons_to_metadata, f)

In [25]:
dedust_jettons_to_metadata['EQAam5Dl28BMaBtXKdDYkT_DekS6a_zKvOVoH0g4SDQcPFdv']

{'mintable': True,
 'total_supply': '500000000000000',
 'admin': {'address': '0:48ddc9e0faa49aa7767069720e67b793e70de744c11bd56b822f5cf891fc48e8',
  'is_scam': False,
  'is_wallet': True},
 'metadata': {'address': '0:1a9b90e5dbc04c681b5729d0d8913fc37a44ba6bfccabce5681f483848341c3c',
  'name': 'Bonk',
  'symbol': 'BONK',
  'decimals': '9',
  'image': 'https://cache.tonapi.io/imgproxy/cOMlJuViiVXDCkAghnyNj7plX8pAZ9pv3WhklvebpTY/rs:fill:200:200:1/g:no/aHR0cHM6Ly9yYXcuZ2l0aHVidXNlcmNvbnRlbnQuY29tL3RvbmtlZXBlci9vcGVudG9uYXBpL21hc3Rlci9wa2cvcmVmZXJlbmNlcy9tZWRpYS90b2tlbl9wbGFjZWhvbGRlci5wbmc.webp'},
 'verification': 'none',
 'holders_count': 6}

In [30]:
list(dedust_jettons_to_metadata.values()).count({'error': 'rate limit: free tier'}), list(dedust_jettons_to_metadata.values()).count(None) 

(464, 0)

### Первые транзакции по адресам токенов

In [32]:
requests.get(ton_url + '/blockchain/accounts/EQD2R36ustW2sAJBlWsfEWn36mds32QPImD5tsZNX4j5N6dw/transactions', params={'sort_order': 'asc', 'limit': '1'}).json()

{'transactions': [{'hash': 'fe6dc0f0f3fdf35984194de9ec24ee92f8d3c630b4a9da4d18b40223ad897137',
   'lt': 44483237000003,
   'account': {'address': '0:f6477eaeb2d5b6b00241956b1f1169f7ea676cdf640f2260f9b6c64d5f88f937',
    'is_scam': False,
    'is_wallet': False},
   'success': True,
   'utime': 1707406074,
   'orig_status': 'nonexist',
   'end_status': 'active',
   'total_fees': 9972616,
   'end_balance': 33394000,
   'transaction_type': 'TransOrd',
   'state_update_old': '90aec8965afabb16ebc3cb9b408ebae71b618d78788bc80d09843593cac98da4',
   'state_update_new': 'e9cb13646398c49be33fdd0e0414cee3ed9e623f124789627d5edcb72a7dacd1',
   'in_msg': {'msg_type': 'int_msg',
    'created_lt': 44483237000002,
    'ihr_disabled': True,
    'bounce': True,
    'bounced': False,
    'value': 250000000,
    'fwd_fee': 14489444,
    'ihr_fee': 0,
    'destination': {'address': '0:f6477eaeb2d5b6b00241956b1f1169f7ea676cdf640f2260f9b6c64d5f88f937',
     'is_scam': False,
     'is_wallet': False},
    'sour

In [51]:
dedust_jettons_to_create_time = {}
dedust_jettons_to_create_time_errors = {}
try:
    for i, token_addr in tqdm.tqdm(enumerate(dedust_jettons_addr_dist), total=len(dedust_jettons_addr_dist)):
        if dedust_jettons_to_create_time.get(token_addr) is None:
            token_transactions = requests.get(
                ton_url + '/blockchain/accounts/' + token_addr + '/transactions', 
                params={"sort_order": "asc", "limit": 100},
                headers={"Authorization": f"Bearer {ton_api_key}"})
            if not token_transactions:
                dedust_jettons_to_create_time_errors[token_addr] = f"bad_request: {token_transactions.text}"
                continue
            token_transactions = token_transactions.json()
            if len(token_transactions['transactions']) == 0:
                dedust_jettons_to_create_time_errors[token_addr] = f"empty transactions"
                continue
            j = 0
            while token_transactions['transactions'][j]['success'] != True:
                if j == 99 or j == len(token_transactions['transactions']) - 1:
                    dedust_jettons_to_create_time_errors[token_addr] = f"no success transactions out of {j+1} first"
                    continue
                j+=1 
            # print(datetime.datetime.fromtimestamp(token_transactions['transactions'][j]['utime']), j)
            # print(a['transactions'][0])
            dedust_jettons_to_create_time[token_addr] = token_transactions['transactions'][j]
except Exception as e:
    print(e)
    with open('dicts/dedust_jettons_to_create_time.pkl', 'wb') as f:
        pickle.dump(dedust_jettons_to_create_time, f)
    with open('dicts/dedust_jettons_to_create_time_errors.pkl', 'wb') as f:
        pickle.dump(dedust_jettons_to_create_time_errors, f)

  0%|          | 0/18135 [00:00<?, ?it/s]

100%|██████████| 18135/18135 [59:15<00:00,  5.10it/s]  


In [52]:
with open('dicts/dedust_jettons_to_create_time.pkl', 'wb') as f:
    pickle.dump(dedust_jettons_to_create_time, f)
with open('dicts/dedust_jettons_to_create_time_errors.pkl', 'wb') as f:
    pickle.dump(dedust_jettons_to_create_time_errors, f)

In [53]:
len(dedust_jettons_to_create_time.keys()), len(dedust_jettons_to_create_time_errors.keys())

(8143, 9992)

In [54]:
dedust_jettons_to_create_time_errors

{'EQDUUeVL3hE4PVEBQKgt5uhvVbdFl6-_y2wMH949RiQcRY0z': 'bad_request: {"error":"rate limit: limit for tier"}',
 'EQD6ZT5B2V0n4QzWH1yBacInRD3cCBU0yc7SUn9xNZR7gdaB': 'bad_request: {"error":"rate limit: limit for tier"}',
 'EQDJsLQ0X2V4EUcUSPJlcLsfnvGpCtAhwoTSKWs8aYn989UJ': 'bad_request: {"error":"rate limit: limit for tier"}',
 'EQAOhLqgW79Kn94PEFr6FxtO1x1KnJrqblHaPr4XrNfiWINm': 'bad_request: {"error":"rate limit: limit for tier"}',
 'EQBqIGgBFOa2CAMS0nMtxf6DOZSjKwFZcRg9wTTcme0sRc4Y': 'bad_request: {"error":"rate limit: limit for tier"}',
 'EQBqGJRj90aixYPsS-iQGzF614-tfkDZaY3iVCxOc0ETq9kY': 'bad_request: {"error":"rate limit: limit for tier"}',
 'EQC0c1UQCIpL7gGfVIiYscIgzOxl02Rr9sVVIo5YRFSBZNJp': 'bad_request: {"error":"rate limit: limit for tier"}',
 'EQBTc1isZGs7mZva7Rcv-dmPQC1QS2iB2Acg2K9V6cm0ttST': 'bad_request: {"error":"rate limit: limit for tier"}',
 'EQBqNo8-lfKRUINFXFkiqPs3hmBdA5SOkx3yJ1qXt3NH-XYb': 'bad_request: {"error":"rate limit: limit for tier"}',
 'EQDUByoXDbCgcKhrQ9UvzpObl9

## Достаем все адреса токенов из всех пулов на Ston.fi

In [126]:
url = stonfi_url + '/pools'
print(f'get {url}')
pools = requests.get(url)
stonfi_tokens = set()
if pools:
    pools = pools.json()['pool_list']
    for i, jetton in enumerate(pools):
        stonfi_tokens.add(jetton['token0_address'])
        stonfi_tokens.add(jetton['token1_address'])
else:
    print(pools)

print(len(stonfi_tokens))

get https://api.ston.fi/v1/pools
10913


In [84]:
pools_tokens = dedust_tokens.union(stonfi_tokens)
print(len(pools_tokens))

23945


## токены с Ston.fi

In [56]:
error_tokens = []
success_tokens = []
for i, token_addr in tqdm.tqdm(enumerate(stonfi_tokens), total=len(stonfi_tokens)):
    token_transactions = requests.get(ton_url + '/blockchain/accounts/' + token_addr + '/transactions', params={"sort_order": "asc", "limit": 100})
    if not token_transactions:
        error_tokens.append({"addr": token_addr, "reason": f"bad_request: {token_transactions}"})
        continue
    token_transactions = token_transactions.json()
    if len(token_transactions['transactions']) == 0:
        error_tokens.append({"addr": token_addr, "reason": f"empty transactions"})
        continue
    j = 0
    while token_transactions['transactions'][j]['success'] != True:
        if j == 99 or j == len(token_transactions['transactions']) - 1:
            error_tokens.append({"addr": token_addr, "reason": f"no success transactions out of {j} first"})
            continue
        j+=1
    # print(datetime.datetime.fromtimestamp(token_transactions['transactions'][j]['utime']), j)
    # print(a['transactions'][0])
    success_tokens.append(token_transactions['transactions'][j])
with open('success_tokens.pkl', 'wb') as f:
    pickle.dump(success_tokens, f)
with open('error_tokens.pkl', 'wb') as f:
    pickle.dump(error_tokens, f)


100%|██████████| 10756/10756 [1:11:26<00:00,  2.51it/s] 


In [ ]:
pickle.load()

In [68]:
data=pd.DataFrame(list(map(lambda x: (x['account']['address'], datetime.datetime.fromtimestamp(x['utime']).year, datetime.datetime.fromtimestamp(x['utime']).month), 
                      success_tokens)), columns=['address', 'year', 'month'])
data.groupby(['year', 'month']).count()


address
year month         
2021 8            1
2022 4            2
     5            1
     6            1
     7            2
     9            3
     10           3
     11           4
     12           5
2023 1            2
     2            3
     3            2
     4            4
     5            4
     6            5
     7            9
     8            8
     9           15
     10          13
     11          13
     12          13
2024 1          758
     2           99
     3         1412
     4         1831
     5         1134

## токены с DeDust с названием

In [105]:
error_tokens = []
success_tokens = []
for i, (token_name, token_addr) in tqdm.tqdm(enumerate(set(dedust_names)), total=len(set(dedust_names))):
    token_transactions = requests.get(ton_url + '/blockchain/accounts/' + token_addr + '/transactions', params={"sort_order": "asc", "limit": 100})
    if not token_transactions:
        error_tokens.append({"addr": token_addr, "reason": f"bad_request: {token_transactions}"})
        continue
    token_transactions = token_transactions.json()
    if len(token_transactions['transactions']) == 0:
        error_tokens.append({"addr": token_addr, "reason": f"empty transactions"})
        continue
    j = 0
    while token_transactions['transactions'][j]['success'] != True:
        if j == 99 or j == len(token_transactions['transactions']) - 1:
            error_tokens.append({"addr": token_addr, "reason": f"no success transactions out of {j} first"})
            continue
        j+=1
    # print(datetime.datetime.fromtimestamp(token_transactions['transactions'][j]['utime']), j)
    # print(token_transactions['transactions'][j])
    success_tokens.append(token_transactions['transactions'][j])
    # if i == 10:
        # break
# with open('success_tokens.pkl', 'wb') as f:
#     pickle.dump(success_tokens, f)
# with open('error_tokens.pkl', 'wb') as f:
#     pickle.dump(error_tokens, f)


100%|██████████| 242/242 [00:54<00:00,  4.41it/s]


In [107]:
data=pd.DataFrame(list(map(lambda x: (x['account']['address'], datetime.datetime.fromtimestamp(x['utime']).year, datetime.datetime.fromtimestamp(x['utime']).month), 
                      success_tokens)), columns=['address', 'year', 'month'])
data.groupby(['year', 'month']).count()


address
year month         
2022 6            1
     7            1
     9            1
     11           3
     12           1
2023 1            5
     3            1
     4            2
     5            1
     6            1
     8            4
     9            2
     10           1
     11           1
     12           4
2024 1           12
     2           13
     3           13
     4           16
     5            2

In [121]:
a = requests.get(stonfi_url + "/stats/pool", params={'since': "2024-05-01T12:34:56", 'until': '2024-05-20T23:59:59'})
all_tokens_may = set(map(lambda x: x['base_symbol'],a.json()['stats'])).union(set(map(lambda x: x['quote_symbol'],a.json()['stats'])))
len(all_tokens_may)

7333

In [122]:
a = requests.get(stonfi_url + "/stats/pool", params={'since': "2024-04-01T00:00:01", 'until': '2024-05-01T00:00:01'})
all_tokens_april = set(map(lambda x: x['base_symbol'],a.json()['stats'])).union(set(map(lambda x: x['quote_symbol'],a.json()['stats'])))
len(all_tokens_april)

5710

In [123]:
a = requests.get(stonfi_url + "/stats/pool", params={'since': "2024-03-01T00:00:01", 'until': '2024-04-01T00:00:01'})
all_tokens_march = set(map(lambda x: x['base_symbol'],a.json()['stats'])).union(set(map(lambda x: x['quote_symbol'],a.json()['stats'])))
len(all_tokens_march)

3306

In [124]:
a = requests.get(stonfi_url + "/stats/pool", params={'since': "2024-02-01T00:00:01", 'until': '2024-03-01T00:00:01'})
all_tokens_feb = set(map(lambda x: x['base_symbol'],a.json()['stats'])).union(set(map(lambda x: x['quote_symbol'],a.json()['stats'])))
len(all_tokens_feb)

1484

In [125]:
a = requests.get(stonfi_url + "/stats/pool", params={'since': "2024-01-01T00:00:01", 'until': '2024-02-01T00:00:01'})
all_tokens_jan = set(map(lambda x: x['base_symbol'],a.json()['stats'])).union(set(map(lambda x: x['quote_symbol'],a.json()['stats'])))
len(all_tokens_jan)

1326

январь - 1326

февраль - 1484 (+158)

март - 3306 (+1822)

апрель - 5710 (+2404)

май - 7333 (+1623)

In [134]:
all_tokens_jan

{'Stay Toned',
 'JCOIN',
 'Tborkkk',
 'TRSHC',
 'CRAB',
 'Duck Dark',
 'PONTON',
 'cMaG',
 'FLOTON',
 'BONTON',
 'Like2',
 'EST',
 'MYCRO',
 'KRYPTON',
 'pcltst',
 'THF',
 'testfest',
 'tBONE',
 'DI',
 'TSN',
 'THub',
 'Vrl',
 'TONY',
 'SquidGrow',
 'donttest',
 'END',
 'TT2',
 'TONOG',
 'STAN',
 'Clown',
 'Tonfi',
 'ALy',
 'OnlyFans',
 'LEGOAI',
 'Starship',
 'JHLC',
 'test2',
 'BOZO',
 'Tshb',
 'gigachadd',
 'DrEgg',
 'OVER',
 'TOOTHLESS',
 'CODE',
 'Pork',
 'NewTON',
 'NTR',
 'TALI',
 'PUTON',
 'TYoshi',
 'SYNTH',
 'TRB',
 'TMyro',
 'WHALE',
 'DOGLABS',
 'MOT',
 'ToNoT',
 'PWNAO',
 'DRE',
 'DOGS',
 'GROYP',
 'pTON-tsTON LP',
 'CUNO',
 'SANIC',
 'ULT',
 'jMAWA',
 'ShS',
 'test3ttt',
 'BOBO',
 'DOGEMEMES',
 'CCS',
 'FlintsTONes',
 'KERMIT',
 'BRB',
 'Pokemon',
 'TGDOG',
 'tPIG',
 'CAKE',
 'DTON',
 '$PORNHUB',
 'f',
 'TONyStark',
 'PUMBA',
 'X',
 'a',
 'DAOXPX',
 'TONEY',
 'TPillz',
 'BANANA',
 'LAVE',
 'bbb',
 'Loony Tons',
 'DOGEMOON',
 'Akimbo',
 'BabyShark',
 'Test',
 'TONKY',
 'FC

In [139]:
DUREV = "EQB02DJ0cdUD4iQDRbBv4aYG3htePHBRK1tGeRtCnatescK0"
a = requests.get(dedust_url + '/gcko/pairs').json()


[{'ticker_id': 'EQBynBO23ywHy_CgarY9NK9FTz0yDsG82PtcbSTQgGoXwiuA:EQC_1YoM8RBixN95lz7odcF3Vrkc_N8Ne7gQi7Abtlet_Efi',
  'base': 'EQBynBO23ywHy_CgarY9NK9FTz0yDsG82PtcbSTQgGoXwiuA',
  'quote': 'EQC_1YoM8RBixN95lz7odcF3Vrkc_N8Ne7gQi7Abtlet_Efi',
  'pool_id': 'EQCpsaUutjz30feCl90WlBXYdT-LhXPl02zKe8L-UWPpfYIG'},
 {'ticker_id': 'EQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAM9c:EQC61IQRl0_la95t27xhIpjxZt32vl1QQVF2UgTNuvD18W-4',
  'base': 'EQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAM9c',
  'quote': 'EQC61IQRl0_la95t27xhIpjxZt32vl1QQVF2UgTNuvD18W-4',
  'pool_id': 'EQDudhi48Qknwmm2CdCxu05M6VXZ7XaLihw-QvMa0pRlESUW'},
 {'ticker_id': 'EQBynBO23ywHy_CgarY9NK9FTz0yDsG82PtcbSTQgGoXwiuA:EQCcLAW537KnRg_aSPrnQJoyYjOZkzqYp6FVmRUvN1crSazV',
  'base': 'EQBynBO23ywHy_CgarY9NK9FTz0yDsG82PtcbSTQgGoXwiuA',
  'quote': 'EQCcLAW537KnRg_aSPrnQJoyYjOZkzqYp6FVmRUvN1crSazV',
  'pool_id': 'EQCb5Yh3R1EuLpT1KnHsE9kPGs41EIbdZ7pBtYndlmVeYPyk'},
 {'ticker_id': 'EQDo_ZJyQ_YqBzBwbVpMmhbhIddKtRP99HugZJ14aFscxi7B:EQBl3gg6AAdjgj

In [143]:
DUREV_ticker_id = ""
for d in a:
    if d['base'] == DUREV:
        print("base", d)
    if d['quote'] == DUREV:
        print("quote", d)
        DUREV_ticker_id = d['ticker_id']


quote {'ticker_id': 'EQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAM9c:EQB02DJ0cdUD4iQDRbBv4aYG3htePHBRK1tGeRtCnatescK0', 'base': 'EQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAM9c', 'quote': 'EQB02DJ0cdUD4iQDRbBv4aYG3htePHBRK1tGeRtCnatescK0', 'pool_id': 'EQCCsJOGdUdSGq0ambJFgSptdHfDPkaQlKlLIKTqtazhhcps'}


In [151]:
requests.get(dedust_url + '/gcko/trades', params={"ticker_id": DUREV_ticker_id, "type": "buy"}).json()

[{'id': 466069620,
  'price': '88.610307149714285714',
  'qty': '3.5',
  'quoteQty': '310.136075024',
  'time': 1716137528},
 {'id': 466070050,
  'price': '88.055648672266917293',
  'qty': '532',
  'quoteQty': '46845.605093646',
  'time': 1716137709},
 {'id': 466070250,
  'price': '87.4978368902',
  'qty': '10',
  'quoteQty': '874.978368902',
  'time': 1716137799},
 {'id': 466071420,
  'price': '87.452062342885714286',
  'qty': '35',
  'quoteQty': '3060.822182001',
  'time': 1716138332},
 {'id': 466071590,
  'price': '87.3962368393',
  'qty': '20',
  'quoteQty': '1747.924736786',
  'time': 1716138409},
 {'id': 466072730,
  'price': '88.564334126',
  'qty': '1',
  'quoteQty': '88.564334126',
  'time': 1716138922},
 {'id': 466072780,
  'price': '88.562264392',
  'qty': '1',
  'quoteQty': '88.562264392',
  'time': 1716138937},
 {'id': 466072860,
  'price': '88.4578672331',
  'qty': '100',
  'quoteQty': '8845.78672331',
  'time': 1716138975},
 {'id': 466073370,
  'price': '88.3538610419381

In [155]:
requests.get(dedust_url + '/pools/' + DUREV + '/trades').json()

[]

In [157]:
oper = requests.get(stonfi_url + "/stats/operations", params={"since": "2024-05-20T00:00:01", "until": "2024-05-20T23:59:59"}).json()

In [162]:
pools = requests.get(stonfi_url + "/stats/pool", params={"since": "2024-05-20T00:00:01", "until": "2024-05-20T23:59:59"}).json()

In [165]:
for pool in pools['stats']:
    if pool['base_id'] == DUREV or pool['quote_id'] == DUREV:
        print(pool)
print(DUREV)

{'pool_address': 'EQCfijjztJQZH1fP53HPj-25AnPg25faTyW-1WKMGW_cWEWx', 'router_address': 'EQB3ncyBUTjZUA5EnFKR5_EnOMI9V1tTEAAPaiU71gc4TiUt', 'url': 'https://app.ston.fi/swap?ft=EQB02DJ0cdUD4iQDRbBv4aYG3htePHBRK1tGeRtCnatescK0&tt=EQCxE6mUtQJKFnGfaROTKOt1lZbDiiX1kCixRv7Nw2Id_sDs', 'base_id': 'EQB02DJ0cdUD4iQDRbBv4aYG3htePHBRK1tGeRtCnatescK0', 'base_name': 'povel durev', 'base_symbol': 'durev', 'quote_id': 'EQCxE6mUtQJKFnGfaROTKOt1lZbDiiX1kCixRv7Nw2Id_sDs', 'quote_name': 'Tether USD', 'quote_symbol': 'USD₮', 'last_price': '0.071891', 'base_volume': '213.951950676', 'quote_volume': '243.374403', 'base_liquidity': '25.887750420', 'quote_liquidity': '1.861120', 'lp_price': '24056.799266672000', 'lp_price_usd': '24056.799266672000', 'apy': '2.9658408794185043'}
{'pool_address': 'EQBUyzmtr5ueiA3RUlWGhhQ-E0j89k5H0IgM8df3zFugWvcR', 'router_address': 'EQB3ncyBUTjZUA5EnFKR5_EnOMI9V1tTEAAPaiU71gc4TiUt', 'url': 'https://app.ston.fi/swap?ft=EQB4gPfG0YtoiJHCMTUuzxP9fKXy9nx6TFh-dcAltrnNpLNo&tt=EQB02DJ0cd

In [167]:
durev_operations = []
for operation in oper['operations']:
    if operation['operation']['pool_address'] == "EQAIp9JcWs87eXXuaTaDXxzQSoN6yt4pffqxy1hXWBBmguqq":
        print(operation)
        durev_operations.append(operation)
len(durev_operations)

{'operation': {'pool_tx_hash': '39ccdba7a6d8eff99880ce65d8df549c30388e07304c1be3c013c486d942ad4f', 'pool_address': 'EQAIp9JcWs87eXXuaTaDXxzQSoN6yt4pffqxy1hXWBBmguqq', 'router_address': 'EQB3ncyBUTjZUA5EnFKR5_EnOMI9V1tTEAAPaiU71gc4TiUt', 'pool_tx_lt': 46628128000001, 'pool_tx_timestamp': '2024-05-20 19:31:28', 'destination_wallet_address': 'EQBSXbLpwD0uYmcdW4t3HW_OQc-POR2L30lRWW9y2na70hDT', 'operation_type': 'swap', 'success': True, 'exit_code': 'swap_ok', 'asset0_address': 'EQCM3B12QK1e4yZSf8GtBRT0aLMNyEsBc_DhVfRRtOEffLez', 'asset0_amount': '-787406554', 'asset0_delta': '-788984524', 'asset0_reserve': '39581517433990', 'asset1_address': 'EQB02DJ0cdUD4iQDRbBv4aYG3htePHBRK1tGeRtCnatescK0', 'asset1_amount': '70000000000', 'asset1_delta': '70000000000', 'asset1_reserve': '3504783617149579', 'lp_token_delta': '0', 'lp_token_supply': '272350661766', 'fee_asset_address': 'EQCM3B12QK1e4yZSf8GtBRT0aLMNyEsBc_DhVfRRtOEffLez', 'lp_fee_amount': '1581100', 'protocol_fee_amount': '788985', 'referral_

615

In [170]:
durev_operations[0]['asset0_info']

{'contract_address': 'EQCM3B12QK1e4yZSf8GtBRT0aLMNyEsBc_DhVfRRtOEffLez',
 'symbol': 'pTON',
 'display_name': 'Proxy TON',
 'image_url': 'https://asset.ston.fi/img/EQCM3B12QK1e4yZSf8GtBRT0aLMNyEsBc_DhVfRRtOEffLez',
 'decimals': 9,
 'kind': 'Jetton',
 'deprecated': False,
 'community': True,
 'blacklisted': False,
 'default_symbol': False}

In [180]:
pd.DataFrame(list(map(lambda x: dict(list(x['operation'].items()) + list(("asset0_info_" + k, v) for k,v in x['asset0_info'].items()) + list(("asset1_info_" + k, v) for k,v in x['asset1_info'].items())) ,durev_operations))).to_excel("durev_ton_stonfi_pool_20.xlsx")

In [168]:
pd.DataFrame(durev_operations)

,operation,asset0_info,asset1_info
0,{'pool_tx_hash': '39ccdba7a6d8eff99880ce65d8df...,{'contract_address': 'EQCM3B12QK1e4yZSf8GtBRT0...,{'contract_address': 'EQB02DJ0cdUD4iQDRbBv4aYG...
1,{'pool_tx_hash': 'cd13f14e2061efee99f498246aa2...,{'contract_address': 'EQCM3B12QK1e4yZSf8GtBRT0...,{'contract_address': 'EQB02DJ0cdUD4iQDRbBv4aYG...
2,{'pool_tx_hash': '3369083d2fb04bc326ff1cfd9584...,{'contract_address': 'EQCM3B12QK1e4yZSf8GtBRT0...,{'contract_address': 'EQB02DJ0cdUD4iQDRbBv4aYG...
3,{'pool_tx_hash': '38edcf81e3d381ef3343f8b449e3...,{'contract_address': 'EQCM3B12QK1e4yZSf8GtBRT0...,{'contract_address': 'EQB02DJ0cdUD4iQDRbBv4aYG...
4,{'pool_tx_hash': '38f92c09801ebd424903373e272c...,{'contract_address': 'EQCM3B12QK1e4yZSf8GtBRT0...,{'contract_address': 'EQB02DJ0cdUD4iQDRbBv4aYG...
...,...,...,...
610,{'pool_tx_hash': '96d361b7ea429111bf713b83f7a6...,{'contract_address': 'EQCM3B12QK1e4yZSf8GtBRT0...,{'contract_address': 'EQB02DJ0cdUD4iQDRbBv4aYG...
611,{'pool_tx_hash': '97e57d4447c06055ef0a3c4c3cf3...,{'contract_address': 'EQCM3B12QK1e4yZSf8GtBRT0...,{'contract_address': 'EQB02DJ0cdUD4iQDRbBv4aYG...
612,{'pool_tx_hash': '987a096616cabf0cba3de2b36a38...,{'contract_address': 'EQCM3B12QK1e4yZSf8GtBRT0...,{'contract_address': 'EQB02DJ0cdUD4iQDRbBv4aYG...
613,{'pool_tx_hash': '98be310dd13fd332dfb597e2e139...,{'contract_address': 'EQCM3B12QK1e4yZSf8GtBRT0...,{'contract_address': 'EQB02DJ0cdUD4iQDRbBv4aYG...
